In [ ]:
#目标检测
#简单的边缘框可以用四个数字（左上+右下 / 右上+左下）定义，精细的怎么做？
# COCO数据集（80个类别）
#代码(属于画图范围)

next(iter())    #next() 从迭代器中取下一个元素，常与iter()结合使用

# 锚框
# 先提出多个被称为锚框的区域(边缘框)，预测内部是否含有关注物体
# 如果是，预测从这个锚框到真是边缘框的偏移
#       用IoU计算相似度(J(A,B))
#       训练时，每个锚框是一个训练样本(一般一张图有上万个)，要么标注成背景(所以会有大量负类样本)，要么关联上一个真实边缘框
#       一张图片进来有9个锚框，就会有9个训练样本，训练时很可能是一张张图片读的
#       被当作背景的锚框点对应的整个行 & 列会被设为背景，不再计算(控制丢弃率，提升泛化能力)
#       去掉相似 / 重复的预测框 ， 使用非极大抑制(NMS)输出 

#   (锚框)代码

### R(Region)-CNN（难实现）
启发式搜索选锚框，SVM对类别分类，L-R模型来预测边缘框偏移  
用最大池化控制形状
### Fast R-CNN
用CNN对整个(Faster的原因)图片抽特征  
用RoI池化对每个锚框生成固定长度特征，直接送入全连接  
精度损失：对小目标影响大
### Faster R-CNN
生成候选锚框步骤交给了NN
Fast手动提框(有监督)，对每个框重复特征提取，开销大，使用RoIPooling  
(快百倍)Faster自提框，共享整张图特征图，效率高；RoIPooling+Full
### Mask RCNN
对像素做标号，用FCN来利用这些信息
### SSD(单发多框检测)
每个像素生成多个以他为中心的多个锚框，通过多分辨率检测提升效果  
底层检测小物体，上层检测大物体
### YOLO
SSD的基础上尽量不重叠锚框(加速)
每个锚框预测B个边缘框

In [ ]:
# SSD实现

### 语义分割
应用：背景虚化，路面分割，实例分割(目标检测进化版本)
如何利用景深信息ladir

In [ ]:
# 实例分割实现

### 转置卷积
可以增大输入的高宽，可以叫反卷积，De-conv / UpConv(相比反卷积，叫转置卷积更准确)
常用于生成任务
本质也是一种卷积(P47.2)

### FCN
用转置卷积层替换CNN最后的全连接层(参数变少降低计算量，适应各种尺寸，保留空间结构信息)，实现每个像素的预测

### 样式迁移
样式图片+内容图片

In [ ]:
# 转置卷积实现

# 样式迁移实现
import torchvision as tv 
import transforms as tf

content_img = 
style_img = 

rgb_mean = torch.tensor([0.485,0.456,0.406])
rgb_std = torch.tensor([0.229,0.224,0.225])

def preprocess(img , image_shape):#图片变成tensor
    transforms = tv.tf.Compose([
        tv.tf.Resize(image_shape),
        tv.tf.ToTensor()，
        tv.tf.Normalize(mean=rgb_mean,std=rgb_std)
    return tf(img).unsqueeze(0)  #在第0维度增加了1维，并且赋值为1；如果是unsqueeze(2)是在第2维维度值赋值为1(而不是数值)
    #若原来张量为[1,2,3],在unsqueeze(0)后变成了[[1,2,3]] ; unsqueeze(1)后变成([[1],[2],[3]])
    ])
def postprocess(img):           #tensor变成图片
    img = img[0].to(rgb_std.device)
    img = torch.clamp(
        img.permute(1,2,0)*rgb_std + rgb_mean,0,1) 
    return tv.tf.ToPILImage()(img) # 创建对象后调用函数
    #将torch中标准化后的图像恢复为可显示形式(反标准化 + 裁剪像素)
    #大或小于max / min的会被截断后变成max / min
    # permute() 改变维度顺序
    # view() 改变形状
#抽取图像特征
pretrained_net = tv.models.vgg19(pretrained = True)
style_layers , content_layers = [0,5,10,19,28],[25]
net = nn.Sequential(*[pretrained_net.features[i] for i in range(max(content_layers+style_layers)+1)])
def extract_features(X,content_layers,style_layers):
    contents = []
    styles = []
    for i in range(len(net)):
        X = net[i](X)
        if i in style_layers:
            styles.append(X)
        if i in content_layers:
            contents.append(X)
        return contents,styles
def get_contents(image_shape,device):
    content_X =
    contents_Y,_ =          #空余“_”表示占位符，即不关心这个值
def get_styles(image_shape,device):
    style_X = 
    _,style_Y = 
    return style_X,style_Y

# 定义损失(如何定义样式类似，匹配两个图片通道的统计信息,这里使用的gram矩阵)
#风格转移的损失函数是内容损失，风格损失，总变化损失的加权和
def content_loss(Y_hat,Y):
    return torch.square(Y_hat - Y.detach()).mean()
def gram(X):
def style_loss(Y_hat,gram_Y):
def tv_loss(Y_hat):             #tv就是图像平滑正则项(惩罚相邻像素的差异)，抑制图像噪声和不必要的高频细节
def compute_loss():

#初始化合成图像
class SynthesizedImage(nn.Module):
    def __init__(self.image_shape,**kwargs):
        super(SynthesizedImage,self).__init__(**kwargs)
        self.weight = nn.Parameter(torch.rand(*image_shape))
    def forward(self):
        return self.weight
def get_inits(X,device,lr,styles_Y):

# 训练
def train()

In [ ]:
.detach()     #返回一个不带梯度，不参与反向传播的新张量，它的数值和原张量相同
x = torch.tensor([2.0], requires_grad=True)
y = x * 3
z = y.detach()
print(z.requires_grad)   # False
print(y.requires_grad)   # True

tensor.pow(exponent)    #将 tensor 中的每个元素都 提升到 exponent 次幂。
x = torch.tensor([2.0, 3.0, 4.0])
y = x.pow(2)
print(y)  # 输出: tensor([ 4.,  9., 16.])

